## Example - electron-ion collision operator test for l=0

This example tests temperature relaxation between electrons and ions due to Coulomb collisions.

This is the v2.0.0 rewrite of the notebook that corresponds to the second test in Section 5.2.2. in the ReMKiT1D paper.

In [ ]:
import numpy as np
import holoviews as hv
import matplotlib.pyplot as plt

import RMK_support as rmk
import RMK_support.common_models as cm

import scipy.optimize

### Wrapper initialization

In [ ]:
rk = rmk.RMKContext()
rk.IOContext = rmk.IOContext(HDF5Dir="./RMKOutput/RMK_ei_coll_test/")

### Grid setup

In [ ]:
xGrid = np.ones(1) # 0D
dv0 = 0.0307
cv = 1.025
vGrid = [dv0/2]
for i in range(1,120):
    vGrid.append(vGrid[i-1]*cv)
lMax = 0 
rk.grid = rmk.Grid(xGrid,np.array(vGrid),lMax,interpretXGridAsWidths=True,interpretVGridAsWidths=True)

### Set temperature derivation option and add electron species

In [ ]:
rk.textbook = rmk.Textbook(rk.grid,[-1,0]) 

rk.species.add(rmk.Species("e",0))
rk.species.add(rmk.Species("D+",-1,atomicA=2.014,charge=1.0))

### Variable initialization

In [ ]:
T0 = 0.8
n0 = 1.0
TInit = T0*np.ones(rk.grid.numX)
nInit = n0 * np.ones(rk.grid.numX)
WInit = 3*nInit*TInit/2

fInit = np.zeros([rk.grid.numX,rk.grid.numH,rk.grid.numV])
for i in range(rk.grid.numX):
    fInit[i,rk.grid.getH(0)-1,:] = (T0*np.pi)**(-1.5) * n0 * np.exp(-rk.grid.vGrid**2/T0) 

f = rmk.Variable("f",rk.grid,data=fInit,isDistribution=True)
W = rmk.Variable("W",rk.grid,derivation=rk.textbook["energyMoment"],derivationArgs=["f"])
n = rmk.Variable("n",rk.grid,derivation=rk.textbook["densityMoment"],derivationArgs=["f"])
zeroVar = rmk.Variable("zeroVar",rk.grid,isDerived=True,inOutput=False)
T = rmk.Variable("T",rk.grid,derivation=rk.textbook["tempFromEnergye"],derivationArgs=["W","n","zeroVar"])
Wi = rmk.Variable("Wi",rk.grid,data=WInit/2)
Ti = rmk.Variable("Ti",rk.grid,derivation=rk.textbook["tempFromEnergyD+"],derivationArgs=["Wi","n","zeroVar"])

rk.variables.add(f,W,n,zeroVar,T,Wi,Ti)

### Adding e-e collision operator model for l = 0

Adding the electron-electron operator here to keep the distribution from deviating from a Maxwellian for the analytical comparison

In [ ]:
rk.models.add(cm.eeCollIsotropic(f,T,n,rk.norms,rk.grid,rk.textbook))

### Adding e-i collision operator model for l = 0

The e-i collision operator for l=0 is implemented in common_models.py and only used here.

In [ ]:
eiCollModel = cm.eiCollIsotropic(rk.grid,rk.textbook,rk.norms,f,T,n,Ti,n,rk.species["D+"],Wi)
rk.models.add(eiCollModel)

### Integrator and timestep options

Simple single step backwards Euler integration

In [ ]:
integrator = rmk.BDEIntegrator("BDE",absTol=10.0,convergenceVars=[W,f,Wi])
integrationStep = rmk.IntegrationStep("BE",integrator)
integrationStep.add(rk.models) 
rk.integrationScheme = rmk.IntegrationScheme(dt=0.1,steps=integrationStep) 
rk.integrationScheme.setFixedNumTimesteps(40000,500) 

Adding Coulomb log diagnostic variable using the extractor manipulator.

In [ ]:
rk.variables.add(rmk.Variable("logLei",rk.grid,isDerived=True))
rk.manipulators.add(rmk.MBDataExtractor("logLei",eiCollModel,eiCollModel.mbData["logLei"]))

#### Generate a LaTeX summary of the ReMKiT1D run 

In [ ]:
rk.generatePDF("Isotropic e-i collision test")

### Create config file

In [ ]:
rk.writeConfigFile()

### Data analysis


In [ ]:
numFiles = 80

#### Loading data

In [ ]:
loadedData = rk.loadSimulation()
dataset=loadedData.dataset

In [ ]:
dataset

In [ ]:
hv.extension('matplotlib')
%matplotlib inline 
plt.rcParams['figure.dpi'] = 150
hv.output(size=80,dpi=150)

### Compare with analytical solution

Note that here we let the Coulomb log vary, so that might introduce another small error compared to the analytical solution. For analytical solution form see Shkarofsky's book Chapter 7-10.

In [ ]:
elCharge = 1.60218e-19
elMass = 9.10938e-31
epsilon0 = 8.854188e-12 #vacuum permittivity 
amu = 1.6605390666e-27 #atomic mass unit
gamma0norm = elCharge**4/(4*np.pi*elMass**2*epsilon0**2)
Ttot = (dataset["T"].data[0,0] + dataset["Ti"].data[0,0])/2 # Plasma temperature
tei0 = 8*gamma0norm  * elMass / (rk.species["D+"].atomicA * amu)*2*dataset["n"].data[0]*rk.norms["density"]*(elMass/(2*elCharge*Ttot*rk.norms["eVTemperature"]))**(3/2)/(3*np.sqrt(np.pi)) 

In [ ]:
tei0 = tei0[0]*rk.norms["time"] # Ratio of time normalization to the temperature relaxation time

In [ ]:
def analyticDeltaT(x,*args):
    return 2*(args[0]**(3/2)-(1+x)**(3/2))/3+2*(np.sqrt(args[0])-np.sqrt(1+x)) + np.log((np.sqrt(args[0])-1)/(np.sqrt(args[0])+1)) - np.log((np.sqrt(1+x)-1)/(np.sqrt(1+x)+1)) - args[1]

In [ ]:
def analyticDeltaTJac(x,*args):

    return - (1+x)**(3/2)/x

In [ ]:
(dataset["T"]+dataset["Ti"])/2

In [ ]:
analyticDT = np.zeros(numFiles+1)
x0 = 1 + 0.5*(dataset["T"].data[0,0] - dataset["Ti"].data[0,0])/Ttot # initial condition (1+xi in Shkarofsky)
tei = np.zeros(numFiles+1)
for i in range(numFiles+1):
    tei[i] = tei0 * dataset["time"].data[i] * dataset["logLei"].data[i,0]
    guessdT = 0.5*(dataset["T"].data[i,0] - dataset["Ti"].data[i,0])/Ttot
    root = scipy.optimize.fsolve(analyticDeltaT,guessdT,args=(x0,tei[i]),fprime=analyticDeltaTJac)
    analyticDT[i] = root[0]*2*Ttot

In [ ]:
deltaT = dataset["T"].data[:,0] - dataset["Ti"].data[:,0]

In [ ]:
analyticCurve = hv.Curve((tei,analyticDT),label="Analytical formula")
numericalScatter = hv.Scatter((tei,deltaT),label="Numerical result").opts(marker="x",color="r",s=5.0)

overlay = analyticCurve*numericalScatter
overlay.opts(xlabel="$t'_{ei}$",ylabel="$\Delta T [10eV]$")

#### Visualising the kinetic over-relaxation

In [ ]:
error = deltaT-analyticDT

errorPlot=hv.Curve((tei,error)).opts(xlabel="$t'_{ei}$",ylabel="$\Delta T_{err} [10eV]$")

In [ ]:
hv.output(fig='pdf')
hv.save(overlay.opts(xlabel="$t'_{ei}$",ylabel="$\Delta T [10eV]$"), 'e-iTempRel.pdf', dpi=144)
hv.save(errorPlot,'e-iOverRel.pdf',dpi=144)